# Atelier DAY 1 :  classer du texte 

L’objectif de cet atelier est d'apprendre à classer du texte dans deux classes : "positif" codé par 1 et "négatif" codé par 0. L'objectif est de
- donner des pistes sur le traitement de données textuelles 
- faire tourner une logistique / bayes naif

Les fichiers IMDB avec lesquels nous travaillons sont disponibles au format python

- `train_NLP.pkl` contient deux objets : les *critiques* et les *labels* associés
- `test_NLP.pkl` contient deux objets : les *critiques* et les *labels* associés

 # Table des matières

[1. Préliminaires ](#basics)<br>
[2. Bases de travail ](#data)<br>
[3. Modélisations ](#model)<br>
[4. Performance](#perf)<br>

<a id='basics'></a>
# 1. Préliminaires 

In [ ]:
import requests
import os

url_train = 'https://stephanegaiffas.github.io/files/formation_cnrs/train_NLP.pkl.gz'
url_test = 'https://stephanegaiffas.github.io/files/formation_cnrs/test_NLP.pkl.gz'
r_train = requests.get(url_train)
r_test = requests.get(url_test)

# Votre chemin. Par defaut celui du notebook
path_data = '.'

with open(os.path.join(path_data, 'train_NLP.pkl.gz'), 'wb') as f:
    f.write(r_train.content)

with open(os.path.join(path_data, 'test_NLP.pkl.gz'), 'wb') as f:
    f.write(r_test.content)

In [ ]:
# Chargement base de travail
import pickle as pkl
import gzip

with gzip.open(os.path.join(path_data, 'train_NLP.pkl.gz'), 'rb') as f:
    train = pkl.load(f)
    
x_train, y_train = train['critiques'], train['labels']

In [ ]:
######### Visualisation des donnees
import pandas as pd
import random

print('Nb de critiques cinema', len(x_train) )
print(pd.DataFrame(y_train)[0].value_counts())
print('')
ind = random.randint(0, 12500)
print(y_train[ind])
print(x_train[ind])

print('')
ind = random.randint(12500,25000)
print(y_train[ind])
print(x_train[ind])

In [ ]:
# Chargement base de test
with gzip.open(os.path.join(path_data, 'test_NLP.pkl.gz'), 'rb') as f:
    test = pkl.load(f)
    
x_test, y_test = test['critiques'], test['labels']

# Comptage
print('Nb de critiques cinema', len(x_test))
print(pd.DataFrame(y_test)[0].value_counts())

<a id='data'></a>
# 2. Bases de travail

Les données brutes *textuelles* nécessitent d'être transformées en *tableau de nombres* pour utiliser les algorithmes. Les choix proposés:

- coder chaque **mot** par un nombre 
    - restreindre le nombre de mots aux **plus fréquents** rencontrés dans la base d'apprentissage
    - standardiser chaque ligne de texte ==> **même nombre de mots** (éventuellement, compléter par des zeros)
    
- compter le **nombre d'occurences** des mots

### Codage des mots 

On transforme la suite de textes en un tableau de variables qualitatives. 

- les lignes sont le numéro de la critique
- la première (ième) colonne est le premier (jième) mot (détecté dans le dictionnaire)

Puis, on code chaque mot. Remarquons que l'ordre d'appartion des mots est préservé.

In [ ]:
########## preprocessing  des data      
# import tensorflow.preprocessing
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

#tuning
max_mots_dico  = 3000           # on prend les 3000 mots les plus frequents du dico
max_len_critique = 200          # on prend les 200 premiers mots de chaque critique qui apparaissent dans le dico

#restriction données pour rapidité
n_train = 25000                  # on restreint notre base ==> faire varier pour montrer l'importance 200 ne marche pas

#transformation
tok = Tokenizer(num_words = max_mots_dico)       # prend les + frequents
tok.fit_on_texts(x_train)                        # construit le dico a partir du corpus. range du + freq au - freq

sequences = tok.texts_to_sequences(x_train)      # code les critiques en entiers
mot_code = tok.word_index                        # donne le codage mot <=> entier
print('il y a %s uniques tokens' % len(mot_code))

xx_train = pad_sequences(sequences, maxlen = max_len_critique)  # formate les critiques codees de longeur maxlen
yy_train = pd.Series(y_train)
print('Shape of critique tensor:', xx_train.shape)
print('Shape of label tensor:', yy_train.shape)

In [ ]:
len(tok.word_index)

In [ ]:
x_train[0]

In [ ]:
xx_train[0]

In [ ]:
## Par souci de rapidité

# melange toutes les lignes des données pour ne pas avoir les neg au debut et les pos a la fin
indices = np.arange(xx_train.shape[0])
np.random.shuffle(indices)
xx_train2 = xx_train[indices]
yy_train2 = yy_train[indices]

# restiction du set des data
xx_train_cod = xx_train2[:n_train]
yy_train_cod = yy_train2[:n_train]

# distribution des labels
print(yy_train_cod.value_counts())
print(yy_train_cod.value_counts() / n_train)

In [ ]:
type(xx_train_cod)

In [ ]:
xx_train_cod

In [ ]:
print('codage par ordre de frequence decroissante', mot_code)

In [ ]:
print('Codage d une critique', [y_train[10]])
print('')
print('le texte est :',x_train[10])
print('')
print('le texte codé est :', xx_train[10])

L'aspect série temporelle est important : on veut avantager les données les plus récentes

- nombre de mots > *max_len_critique* ==> tronquage de la séquence au début
- nombre de mots < *max_len_critique* ==> ajoût de zéros au début

In [ ]:
# A LAISSER FAIRE sur TEST : attention le dico est deja donne ==> ne pas re apprendre !
sequences = tok.texts_to_sequences(x_test)
# code les critiques en entiers avec le meme tokenizer
xx_test_cod = pad_sequences(sequences, maxlen = max_len_critique)
# transforme les critiques en suite d entiers de longeur maxlen

yy_test_cod = pd.Series(y_test)
print('Shape of critique tensor:', xx_test_cod.shape)
print('Shape of label tensor:', yy_test_cod.shape)

# distribution des labels
print(yy_test_cod.value_counts())
print(yy_test_cod.value_counts() / len(yy_test_cod))

### Nombre d'occurences

On transforme la suite de textes en un tableau de variables quantitatives

- les lignes sont le numéro de la critique
- la première (ième) colonne est 
    - 0 si le premier mot du dictionnaire n'apparait pas
    - TF * IDF ou TF = term frequency = frequence d'apparition du mot dans le document et IDF=log de l'inverse de la proportion de documents ou le terme apparait

Remarquons que l'ordre d'apparition des mots est oublié puisque les colonnes sont dans l'ordre des mots du dictionnaire.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, analyzer='word', 
                       stop_words='english')

# construction du dictionnaire adapté à TRAIN et calcul des occurences
xx_train_occ = vect.fit_transform(x_train) 
yy_train_occ = pd.Series(y_train)

In [ ]:
print(xx_train_occ[0:1000, 0:3])
# en ligne : les critiques
# en colonne : les mots du plus fréquent au moins fréquent dans le corpus

In [ ]:
type(xx_train_occ)

In [ ]:
xx_train.shape

Les critiques n'utilisant qu'un très petit nombre de mots du dictionnaire, la matrice d'occurence est creuse ==> l'affichage ne montre que les termes non nuls.

In [ ]:
# attention: 
# ne pas re-construire un dictionnaire (different!), calculer les occurences avec le dictionnaire construit sur TRAIN
xx_test_occ = vect.transform(x_test)
yy_test_occ = pd.Series(y_test)

### Questions que l'on se pose 

- Vaut-il mieux coder les mots ou utiliser le nombre d'occurence ?
- La modélisation logistique est-elle performante ?

In [ ]:
path_travail = './'

# On sauve tout
with open(os.path.join(path_travail, 'imdb_train_cod.pkl'), 'wb') as f:
    pkl.dump({'features': xx_train_cod, 'labels': yy_train_cod}, f)

with open(os.path.join(path_travail, 'imdb_test_cod.pkl'), 'wb') as f:
    pkl.dump({'features': xx_test_cod, 'labels': yy_test_cod}, f)

with open(os.path.join(path_travail, 'imdb_train_occ.pkl'), 'wb') as f:
    pkl.dump({'features': xx_train_occ, 'labels': yy_train_occ}, f)

with open(os.path.join(path_travail, 'imdb_test_occ.pkl'), 'wb') as f:
    pkl.dump({'features': xx_test_occ, 'labels': yy_test_occ}, f)

<a id='model'></a>
# 2. Modélisations 

Nous proposons d'évaluer les performance de l'algorithme **Bayes Naif** en prenant **la logistique L1 pénalisée** comme benchmark (tunée par défaut). 

La modélisation se fait en trois temps :
- on sépare les données : LEARN / TEST ==> déjà fait
- on apprend le modèle sur TRAIN
- on ré-applique le modèle sur TEST

In [ ]:
# Modèle avec pénalisation L1 <== tuning par défaut

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

x = xx_train_cod.copy()
y = yy_train_cod.copy()

# définition du modèle
logreg = LogisticRegression(penalty='l1', solver='liblinear')
# apprentissage
logreg.fit(x, y)                      

x = xx_test_cod.copy()
y = yy_test_cod.copy()

# Prédiction
res_test = logreg.predict(x)           
temp = pd.Series(res_test, index=y.index)
pred_test = pd.concat([y, temp], axis=1) 
pred_test.rename(columns={1: 'class'}, inplace=True)
pred_test.rename(columns={0: 'y'}, inplace=True)

# Sauver les scores pour log pénalisation par defaut
class_test_L_cod = pred_test.copy()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

plt.figure(figsize=(14, 4))
plt.stem(logreg.coef_.ravel(), use_line_collection=True)
plt.title('Logistic regression coefficients', fontsize=16)
_ = plt.yticks(fontsize=14)

In [ ]:
from sklearn.naive_bayes import MultinomialNB,  BernoulliNB

x = xx_train_cod.copy()
y = yy_train_cod.copy()

NB = MultinomialNB()
NB.fit(x,y)

x = xx_test_cod.copy()
y = yy_test_cod.copy()
# Prédiction
pred_test = NB.predict(x)

# Affichage
temp = pd.Series(pred_test, index=y.index)
pred_test = pd.concat([y, temp], axis=1)
pred_test.rename(columns={0: 'y'}, inplace=True)
pred_test.rename(columns={1: 'class'}, inplace=True)

# Sauver les scores pour NB
class_test_NB_cod = pred_test.copy()

In [ ]:
# Modèle avec pénalisation L1 <== tunning par défaut
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV

x = xx_train_occ.copy()
y = yy_train_occ.copy()

# définition du modèle
logreg = LogisticRegression(penalty='l1', solver='liblinear')
# apprentissage
logreg.fit(x, y)                      

x = xx_test_occ.copy()
y = yy_test_occ.copy()
# Prédiction
res_test = logreg.predict(x)           

# Affichage
temp = pd.Series(res_test, index=y.index)
pred_test = pd.concat([y, temp], axis=1)
pred_test.rename(columns={0: 'y'}, inplace=True)
pred_test.rename(columns={1: 'class'}, inplace=True)

# Sauver les scores pour log pénalisation par defaut
class_test_L_occ = pred_test.copy()

In [ ]:
logreg.n_iter_

In [ ]:
from sklearn.naive_bayes import MultinomialNB,  BernoulliNB

x = xx_train_occ.copy()
y = yy_train_occ.copy()

NB = MultinomialNB()
NB.fit(x,y)

x = xx_test_occ.copy()
y = yy_test_occ.copy()

# Prédiction
pred_test = NB.predict(x)

# Affichage
temp = pd.Series(pred_test, index=y.index)
pred_test = pd.concat([y, temp], axis=1)
pred_test.rename(columns={0: 'y'}, inplace=True)
pred_test.rename(columns={1: 'class'}, inplace=True)

# Sauver les scores pour NB
class_test_NB_occ = pred_test.copy()

<a id='perf'></a>
# 3. PERFORMANCES

La performance pour le problème de classification s'analyse sur la base TEST

- Matrice de confusion (qui donne les deux erreurs et la puissance)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

m_test = class_test_L_cod.copy()

print('Sur le test...')
print(classification_report(m_test['y'], m_test['class']))
print(confusion_matrix(m_test['y'], m_test['class']))

In [ ]:
m_test = class_test_NB_cod.copy()

print('Sur le test...')
print(classification_report(m_test['y'], m_test['class']))
print(confusion_matrix(m_test['y'], m_test['class']))

In [ ]:
m_test = class_test_L_occ.copy()

print('Sur le test...')
print(classification_report(m_test['y'], m_test['class']))
print(confusion_matrix(m_test['y'], m_test['class']))


In [ ]:
m_test = class_test_NB_occ.copy()

print('Sur le test...')
print(classification_report(m_test['y'], m_test['class']))
print(confusion_matrix(m_test['y'], m_test['class']))